In [1]:
import sys,os,datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import fbeta_score
from __future__ import print_function

In [2]:
print(np.__version__);
print(pd.__version__);

1.12.1
0.19.2


In [3]:
import  cv2 as cv
cv.__version__

'3.2.0'

In [4]:
sys.path.append('../Python')
from helper import formFH, paths_input

In [5]:
#from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential,save_model,load_model
from keras.layers import Convolution2D, MaxPooling2D, Convolution1D, MaxPooling1D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers.normalization import BatchNormalization
import keras.optimizers
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [6]:
keras.__version__

'2.0.4'

In [7]:
trLabels,trDirTIF,trDirJPG,teDirTIF,teDirJPG = paths_input()
trDirI = trDirTIF
teDirI = teDirTIF
trWork, teWork = '../Work/Train', '../Work/Test'
trDirI,teDirI, trWork, teWork

('../Data/train-tif-v2',
 '../Data/test-tif-v2',
 '../Work/Train',
 '../Work/Test')

In [8]:
labels_df = pd.read_csv(trLabels)
labels_df.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [9]:
# Build list with unique labels
label_list = []
for tag_str in labels_df.tags.values:
    labels = tag_str.split(' ')
    for label in labels:
        if label not in label_list:
            label_list.append(label)

In [10]:
# Add onehot features for every label
for label in label_list:
    labels_df[label] = labels_df['tags'].apply(lambda x: 1 if label in x.split(' ') else 0)
    #labels_df[label].astype(np.int8)
# Display head
#labels_df.head()


In [11]:
weather_labels = ['clear', 'partly_cloudy', 'haze', 'cloudy']
land_labels = ['primary', 'agriculture', 'water', 'cultivation', 'habitation' ]
rare_labels = [l for l in label_list if labels_df[label_list].sum()[l] < 2000]
#rare_labels              

In [12]:
labels = label_list; #weather_labels;
nameList =labels_df[labels_df[labels].sum(axis=1)>0].image_name.tolist(); len(nameList)
labelList=labels_df[labels_df[labels].sum(axis=1)>0][labels].as_matrix();
labelList[:6,:]
#labels_df.head()

array([[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [13]:
def formImExt (nf, resize=(32,32), printOK=False) :
    nx = None
    try : 
        ni = cv.imread(nf,-1); 
        if (ni is not None) :
            if not ((ni.shape[2]==3) or (ni.shape[2]==4)) and printOK : print('----- error ---- shape:',ni.shape,nf)
            if (ni.shape[2]==3) :
                nx = cv.resize(ni,resize)
            if (ni.shape[2]==4) :
                #r,g,b,n = ni[:,:,2],ni[:,:,1],ni[:,:,0],ni[:,:,3]
                r,g,b,n = cv.resize(ni[:,:,2],resize),cv.resize(ni[:,:,1],resize),cv.resize(ni[:,:,0],resize),cv.resize(ni[:,:,3],resize)
                dv,dw   = np.divide((r-n),(r+n+0.01)), np.divide((g-n),(g+n+0.01))
                nx      = np.array([r,g,b,n,dv,dw]).T; 
    except BaseException as e :
        print(nf,e); nx = None;
    
    if nx is None and printOK : 
        print('------ None:',nf); nx = None
        
    return(nx)

In [161]:
trX, trY, i, size = [],[], 0, len(nameList)
print(datetime.datetime.now(),len(nameList),size)
for nn in nameList[0:size] :
    nf = os.path.join(trDirTIF,nn+".tif");
    nx = formImExt(nf,resize=(64,64))
    if (nx is not None) :
        #rr=np.save(os.path.join(trWork,nn+".npy"),nx);
        #trX.append(nn+".npy")
        trX.append(nx)
        trY.append(True)
    else : 
        trY.append(False)
    i += 1
    if (i%5000==0) : print(datetime.datetime.now(),"\t",i,"\t",nn)
    #print(nn.shape)

print(datetime.datetime.now())
trX = np.array(trX);
trY = labelList[trY];
print(len(nameList),trX.shape,trY.shape)
#trXY=pd.DataFrame(trY); trXY['name']=trX; trXY.head()
#trXY.to_pickle(os.path.join(trWork,"listFiles.pkl"))

2017-05-17 01:13:07.494771 40479 40479
2017-05-17 01:16:37.692809 	 5000 	 train_4999
2017-05-17 01:19:26.533243 	 10000 	 train_9999
2017-05-17 01:22:03.474690 	 15000 	 train_14999
2017-05-17 01:24:39.649915 	 20000 	 train_19999
2017-05-17 01:27:09.540339 	 25000 	 train_24999
2017-05-17 01:29:38.237389 	 30000 	 train_29999
2017-05-17 01:32:05.531116 	 35000 	 train_34999
2017-05-17 01:34:34.413286 	 40000 	 train_39999
2017-05-17 01:34:47.681317
40479 (40479, 64, 64, 6) (40479, 17)


In [162]:
np.save('../Data-Keras/train-model-2D-64x64-v2-XX.npy',trX)
np.save('../Data-Keras/train-model-2D-64x64-v2-YY.npy',trY)

In [160]:
#del(trOX); del(trOY);

In [37]:
trX = np.load('../Data-Keras/train-model-2D-64x64-v2-XX.npy')
trY = np.load('../Data-Keras/train-model-2D-64x64-v2-YY.npy')
print(trX.shape,trY.shape)

(40479, 64, 64, 6) (40479, 17)


In [14]:
import keras.backend as K
def fbeta_pred(y_true, y_pred, beta=2.0, OK1=0.2, eps=0.000001, printOK=False):
    beta2 = beta*beta
    yy_true = K.round(y_true)
    #yy_pred = K.round(y_pred+(0.5-OK1))
    yy_pred = K.round(y_pred)
    tp, tp_fp, fn = K.sum((yy_pred*yy_true)), K.sum(yy_true), K.sum((K.abs(yy_pred*(yy_true-1.0))))
    precision, recall = tp/(tp_fp+eps), tp/(tp+fn+eps) 
    fbeta = (1+beta2)*(precision*recall)/(beta2*precision+recall+eps)
    ##if fbeta>1.0 : fbeta = 1.0;
    if printOK :
        print('ten true ',K.get_value(yy_true))
        #print('ten pred ',y_pred)
        print('ten roun ',K.get_value(yy_pred))
        print(' pre=',K.get_value(precision),' recall=',K.get_value(recall),' tp=',
              K.get_value(tp),' fn=',K.get_value(fn),' tp+fp=',K.get_value(tp_fp))
    return(fbeta)

In [15]:
#
# model Krizevsky 2012
#
#    input(224x224)
#    c2d(96,(11,11),strides=(4,4))
#    c2d(256,(5,5))
#    c2d(384,(3,3))
#    c2d(384,(3,3))
#    c2d(256,(3,3))
#    maxp(2,2)
#    flaten()
#    dense(4096)
#    dense(4096)
#    dense(output(1000))
#
#
def Kriz2012 ( model, iSize, rSize ) :
    i1,i2,i3 = iSize
    model.add(BatchNormalization(input_shape=(i1,i2,i3)))
    
    model.add(Convolution2D(96,(11,11),strides=(4,4),activation='relu'))
    model.add(Convolution2D(256,(5,5),activation='relu'))
    model.add(Convolution2D(384,(3,3),activation='relu'))
    model.add(Convolution2D(384,(3,3),activation='relu'))
    model.add(Convolution2D(256,(3,3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(4096,activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(4096,activation='relu'))
    model.add(Dropout(0.25))
    
    model.add(Dense(rSize,activation='sigmoid'))

    return(model)

In [16]:
#
# model Krizevsky 2012
#
#    input(224x224)
#    c2d(96,(11,11),strides=(4,4))
#    c2d(256,(5,5))
#    c2d(384,(3,3))
#    c2d(384,(3,3))
#    c2d(256,(3,3))
#    maxp(2,2)
#    flaten()
#    dense(4096)
#    dense(4096)
#    dense(output(1000))
#
# cut version
#
#
def Kriz2012c ( model, iSize, rSize ) :
    i1,i2,i3 = iSize
    model.add(BatchNormalization(input_shape=(i1,i2,i3)))
    
    model.add(Convolution2D(96,(11,11),strides=(4,4),activation='relu'))
    model.add(Convolution2D(256,(5,5),activation='relu'))
    model.add(Convolution2D(384,(3,3),activation='relu'))
    model.add(Convolution2D(384,(3,3),activation='relu'))
    model.add(Convolution2D(256,(3,3),activation='relu'))
    model.add(Dropout(0.25))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(4096,activation='relu'))
    model.add(Dropout(0.5))
    ##model.add(Dense(2048,activation='relu'))
    ##model.add(Dropout(0.25))
    #model.add(Dense(4096,activation='relu'))
    #model.add(Dropout(0.25))
    
    model.add(Dense(rSize,activation='sigmoid'))

    return(model)

In [19]:
#
# try model Krizevsky 2012
#
#iSize = (trX.shape[1],trX.shape[2],trX.shape[3])
iSize = (64,64,6)
#rSize = trY.shape[1]
rSize = 17
model = Sequential()
model = Kriz2012c(model,iSize,rSize)
sgd = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer='adam', #sgd, #"adam", #'rmsprop',
              metrics=[fbeta_pred]) #['accuracy',fbeta_pred]) #['accuracy'])

In [20]:
# badly don't work: fail customs metric # model=load_model('../Data-Keras/basin02-loop-all.h5') ## verify load from v1 version
#model.load_weights('../Data-Keras/train-model-020-1D-2-v2-loop-weights.h5') ## verify load weights from v1 version

In [21]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 64, 64, 6)         24        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 96)        69792     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 256)       614656    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 384)         885120    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 6, 6, 384)         1327488   
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 4, 4, 256)         884992    
_________________________________________________________________
dropout_1 (Dropout)          (None, 4, 4, 256)         0         
__________

In [ ]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        #shear_range=0.2,
        #zoom_range=0.2,
        #rotation_range=10,
        #samplewise_center=True,
        ##featurewise_center=True,
        ##featurewise_std_normalization=True,
        ##samplewise_std_normalization=True,
        ##zca_whitening=True,
        #vertical_flip=True,
        #horizontal_flip=True
        )

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(
        #samplewise_center=True,
        ##featurewise_center=True,
        ##featurewise_std_normalization=True,
        #samplewise_std_normalization=True,
        ##zca_whitening=True,
        rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow(xTrain[0:10000],yTrain[0:10000],
        #target_size=(h,w),
        #color_mode='grayscale',
        batch_size=32,)
        #save_to_dir='../Keras/Temp',
        #shuffle=False,
        #class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow(xTest[0:1000],yTest[0:1000],
        #target_size=(h,w),
        #color_mode='grayscale',
        batch_size=32,)
        #save_to_dir='../Keras/Temp',
        #shuffle=False,
        #class_mode='binary')

In [167]:
print('time.begin :',datetime.datetime.now())
#hist_old=hist.history
early_stopping = EarlyStopping(monitor='val_loss', patience=15)
hist = model.fit(trX,trY,
         epochs=100, #10,
         batch_size=256,
         validation_split=0.20,
         callbacks=[early_stopping],
         #class_weight=rrd,
         verbose=2
         )
        #samples_per_epoch=200)
        #validation_data=validation_generator,
        #initial_epoch=start,
        #nb_val_samples=2048
         #)
print('time.end   :',datetime.datetime.now())

time.begin : 2017-05-17 01:43:12.199873
Train on 32383 samples, validate on 8096 samples
Epoch 1/100
77s - loss: 5.6902 - fbeta_pred: 0.6457 - val_loss: 5.4766 - val_fbeta_pred: 0.7209
Epoch 2/100
33s - loss: 5.2851 - fbeta_pred: 0.7625 - val_loss: 5.2076 - val_fbeta_pred: 0.7948
Epoch 3/100
33s - loss: 5.1658 - fbeta_pred: 0.7773 - val_loss: 5.1337 - val_fbeta_pred: 0.8053
Epoch 4/100
33s - loss: 5.0314 - fbeta_pred: 0.7957 - val_loss: 5.0433 - val_fbeta_pred: 0.8053
Epoch 5/100
33s - loss: 4.9689 - fbeta_pred: 0.8064 - val_loss: 4.9210 - val_fbeta_pred: 0.8231
Epoch 6/100
33s - loss: 4.9093 - fbeta_pred: 0.8106 - val_loss: 4.8984 - val_fbeta_pred: 0.8213
Epoch 7/100
33s - loss: 4.8698 - fbeta_pred: 0.8154 - val_loss: 4.9380 - val_fbeta_pred: 0.8276
Epoch 8/100
33s - loss: 4.8304 - fbeta_pred: 0.8168 - val_loss: 4.8110 - val_fbeta_pred: 0.8268
Epoch 9/100
33s - loss: 4.7907 - fbeta_pred: 0.8230 - val_loss: 4.7901 - val_fbeta_pred: 0.8439
Epoch 10/100
33s - loss: 4.7497 - fbeta_pred: 0

In [251]:
save_model(model,'../Data-Keras/train-model-2D-2-v2-all.h5')
model.save_weights('../Data-Keras/train-model-2D-2-v2-weights.h5')  # save weights after training

In [54]:
print(model.evaluate(trX,trY,verbose=2))
trP = model.predict(trX, batch_size=128)
print(trY.shape)
print(trP.shape)
print('fbeta_score=',fbeta_score(trY, np.array(trP) > 0.2, beta=2, average='samples'))
print('fbeta_pred =',K.get_value(fbeta_pred(trY.astype(np.float64),trP.astype(np.float64))))

[4.3339444657595294, 0.71995355617081491, 0.87297436628756564]
(40479, 17)
(40479, 17)
fbeta_score= 0.900652762317
fbeta_pred = 0.872814040159


In [168]:
print(model.evaluate(trX,trY,verbose=2))
trP = model.predict(trX, batch_size=512)
print(trY.shape)
print(trP.shape)
print('fbeta_score=',fbeta_score(trY, np.array(trP) > 0.2, beta=2, average='samples'))
print('fbeta_pred =',K.get_value(fbeta_pred(trY.astype(np.float64),trP.astype(np.float64))))

[4.2616619400457534, 0.88562342545511386]
(40479, 17)
(40479, 17)
fbeta_score= 0.908520702902
fbeta_pred = 0.885392335893


In [286]:
#hist_old=hist.history

In [252]:
rr, rrx = [], []; trP = model.predict(trX, batch_size=512)
for i in range(trP.shape[1]) :
    xx = [];
    trYY = trY[:,i].astype(np.float64)
    trPY = trP[:,i]
    for ii in range(100) :
        trPP = (trPY>0.01*ii).astype(np.float64)
        #x = fbeta_score(trY[:,0], np.array(trP[:,0] > 0.1*ii), beta=2, average='samples')
        x = K.get_value(fbeta_pred(trYY,trPP))
        xx.append(x)
    rrr = np.array(xx).argmax();
    rr.append(rrr)
    rrx.append(xx[rrr])
    print(datetime.datetime.now(),i,'-->',rrr,xx[rrr],xx[(rrr-1):(rrr+2)])
    #print(xx);
    #plt.plot(np.array(xx)); plt.show()
trM = np.array(rr)/100.0;

2017-05-15 07:45:16.245791 0 --> 67 0.703723037192 [0.70136893207458639, 0.70372303719187324, 0.70255697093345204]
2017-05-15 07:48:05.845385 1 --> 74 0.980069323901 [0.97993962183479821, 0.98006932390084267, 0.97997932676465371]
2017-05-15 07:51:03.555375 2 --> 60 0.830832805721 [0.83066919227432678, 0.83083280572116081, 0.83012642884749333]
2017-05-15 07:54:07.692849 3 --> 73 0.95596621068 [0.95530247952573877, 0.9559662106804423, 0.95587193006934856]
2017-05-15 07:57:17.654396 4 --> 37 0.771627389839 [0.77128836874582085, 0.77162738983933221, 0.77157084802598352]
2017-05-15 08:00:35.571387 5 --> 28 0.612647945213 [0.6089740900094327, 0.61264794521337496, 0.60631829415658134]
2017-05-15 08:04:02.404595 6 --> 40 0.711618933993 [0.70809511238058165, 0.71161893399277543, 0.71149522396880938]
2017-05-15 08:07:36.162990 7 --> 38 0.549023191429 [0.54805400909065172, 0.54902319142904643, 0.54874345592600848]
2017-05-15 08:11:16.278601 8 --> 20 0.24951985087 [0.24604540154654581, 0.249519850

In [170]:
save_model(model,  '../Data-Keras/train-model-2D-2-v2-loop-all.h5')
model.save_weights('../Data-Keras/train-model-2D-2-v2-loop-weights.h5')  # save weights after training

In [22]:
model.load_weights('../Data-Keras/train-model-2D-2-v2-loop-weights.h5') ## verify load weights from v1 version

In [23]:
# Forming output dataset for predicting --> trOX, trOY
del(trX)
del(trY)

NameError: name 'trX' is not defined

In [30]:
nameAsk = os.listdir(teDirI); print(len(nameAsk))
trOX, trOY, i, ii, size = [], [], 0, 0, len(nameAsk)
print(datetime.datetime.now(),len(nameAsk),size)
for nn in nameAsk[0:size] :
    nf = os.path.join(teDirTIF,nn);
    nx = formImExt(nf,resize=(64,64))
    if (nx is not None) :
        trOX.append(nx)
        trOY.append(nn)
    i += 1
    if (i%10000==0) and (i>1) :
        if (ii==0) :
            trOX = np.array(trOX);
            trP = model.predict(trOX, batch_size=512); 
        else :
            trOX = np.array(trOX);
            trP = np.vstack([trP,model.predict(trOX, batch_size=512)]); 
        trOX,ii = [],ii+1;
    if (i%5000==0) : print(datetime.datetime.now(),"\t",ii,'\t',i,"\t",nn,'\t',(trP.shape if ii>0 else ""))

if (len(trOX)>0) :
    if (ii==0) :
        trOX = np.array(trOX);
        trP = model.predict(trOX, batch_size=512); 
    else :
        trOX = np.array(trOX);
        trP = np.vstack([trP,model.predict(trOX, batch_size=512)]); 
    trOX,ii = [],ii+1;
    
print(datetime.datetime.now())

#assert (size!=len(trOY)), "Wrong files {} != {}".format(size,len(trOY))

61191
2017-05-17 03:09:48.246618 61191 61191
2017-05-17 03:11:58.205335 	 0 	 5000 	 test_15649.tif 	 
2017-05-17 03:14:10.807284 	 1 	 10000 	 test_32423.tif 	 (10000, 17)
2017-05-17 03:16:13.438699 	 1 	 15000 	 test_36264.tif 	 (10000, 17)
2017-05-17 03:18:21.598552 	 2 	 20000 	 test_10894.tif 	 (20000, 17)
2017-05-17 03:20:28.583399 	 2 	 25000 	 file_9502.tif 	 (20000, 17)
2017-05-17 03:22:35.660930 	 3 	 30000 	 file_14128.tif 	 (30000, 17)
2017-05-17 03:24:39.034604 	 3 	 35000 	 file_17389.tif 	 (30000, 17)
2017-05-17 03:26:47.299834 	 4 	 40000 	 test_19356.tif 	 (40000, 17)
2017-05-17 03:28:49.710308 	 4 	 45000 	 test_10134.tif 	 (40000, 17)
2017-05-17 03:30:57.184685 	 5 	 50000 	 test_24243.tif 	 (50000, 17)
2017-05-17 03:33:03.187887 	 5 	 55000 	 test_7049.tif 	 (50000, 17)
2017-05-17 03:35:12.862152 	 6 	 60000 	 file_10482.tif 	 (60000, 17)
2017-05-17 03:35:42.990272


In [31]:
#trOX = np.array(trOX);
trOY = np.array([os.path.splitext(x)[0] for x in trOY]);
print(len(nameAsk),trP.shape,trOY.shape)

61191 (61191, 17) (61191,)


In [ ]:
# Saving & Loading

In [26]:
np.save('../Data-Keras/test-basin-2D-64x64-OX-tif-v2.npy',trOX)
np.save('../Data-Keras/test-basin-2D-64x64-OY-tif-v2.npy',trOY)

In [27]:
trOX = np.load('../Data-Keras/test-basin-2D-64x64-OX-tif-v2.npy')
trOY = np.load('../Data-Keras/test-basin-2D-64x64-OY-tif-v2.npy')
print(trOX.shape,trOY.shape)

(61191, 32, 32, 6) (61191,)


In [93]:
rrx
rrd=dict()
for i in range(len(rrx)) :
    rrd[i]=rrx[i]
rrd

NameError: name 'rrx' is not defined

In [258]:
#Forming result

In [94]:
#trP = model.predict(trOX, batch_size=512); 
trP.shape, trP[-1]

((61191, 17), array([  2.31444892e-02,   9.79294121e-01,   2.86209971e-01,
          9.69578385e-01,   3.21317136e-01,   3.53857726e-02,
          1.92450643e-01,   1.09134607e-01,   5.12446323e-03,
          7.43655721e-04,   2.26537101e-02,   7.24586775e-04,
          1.53525993e-02,   2.39399401e-03,   2.09098756e-02,
          2.87257116e-02,   1.37278186e-02], dtype=float32))

In [32]:
trM=np.array([0.0]*len(labels)); print(trM.shape)
wr = [labels.index(i) for i in weather_labels];
tt=trM; trM[:]=0.5
trM[np.array(wr)] = 2.0
print(wr,'\n',labels,'\n',trM.tolist())

(17,)
[3, 10, 0, 9] 
 ['haze', 'primary', 'agriculture', 'clear', 'water', 'habitation', 'road', 'cultivation', 'slash_burn', 'cloudy', 'partly_cloudy', 'conventional_mine', 'bare_ground', 'artisinal_mine', 'blooming', 'selective_logging', 'blow_down'] 
 [2.0, 0.5, 0.5, 2.0, 0.5, 0.5, 0.5, 0.5, 0.5, 2.0, 2.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]


In [33]:
#trP = model.predict(trX, batch_size=512); trP=K.get_value(trP)
res = []

for i in range(trP.shape[0]) :
    trPP = [weather_labels[trP[i,wr].argmax()]] + [labels[ii] for ii in range(len(labels)) if (trP[i,ii]>trM[ii])];
    pp   = ' '.join(trPP)
    ##if (pp=="") : print(trY[i])
    res.append([trOY[i],pp])

res.sort(cmp=lambda x,y: cmp(int(x[0].partition('_')[2]),int(y[0].partition('_')[2])) if (x[0].partition('_')[0]==y[0].partition('_')[0]) else cmp(y[0].partition('_')[0],x[0].partition('_')[0]))
#print(res[4:8],'\n',res[-4:])
print(res[0],'\n',res[-1],'\n',trM,trP[0])

['test_0', 'clear primary'] 
 ['file_20521', 'clear primary agriculture habitation road'] 
 [ 2.   0.5  0.5  2.   0.5  0.5  0.5  0.5  0.5  2.   2.   0.5  0.5  0.5  0.5
  0.5  0.5] [  1.94645324e-03   9.97888982e-01   8.46054684e-03   9.90010440e-01
   1.23228263e-02   2.39327317e-03   8.05634167e-03   4.15086700e-03
   1.36539020e-04   9.33622869e-05   7.56200310e-03   1.45730678e-06
   3.36432975e-04   4.43703175e-06   1.29225282e-02   6.47825329e-03
   4.82078525e-04]


In [35]:
print(labels)
#print(trM.tolist())
#np.round(trP[4:11,:])

['haze', 'primary', 'agriculture', 'clear', 'water', 'habitation', 'road', 'cultivation', 'slash_burn', 'cloudy', 'partly_cloudy', 'conventional_mine', 'bare_ground', 'artisinal_mine', 'blooming', 'selective_logging', 'blow_down']


In [36]:
rrr=pd.DataFrame(res,columns=['image_name','tags']); rrr.head(); 
suffixDT = (datetime.datetime.now()).strftime('%Y-%m-%d-%H-%M-%S'); print(suffixDT)
rrr.to_csv('../Result/vss'+suffixDT+'.csv',index=False);

2017-05-17-03-38-07
